## Getting Started

Attempted walkthrough of the [getting started](https://docs.aws.amazon.com/kinesisanalytics/latest/java/get-started-exercise.html) tutorial for KDA-J

In [ ]:
import boto3

kda_client = boto3.client('kinesisanalyticsv2')
kinesis = boto3.client('kinesis')
iam = boto3.client('iam')

### Input and Output Streams

In [ ]:
cis = kinesis.create_stream(
    StreamName='ExampleInputStream',
    ShardCount=1
)

print(cis)

In [ ]:
cos = kinesis.create_stream(
    StreamName='ExampleOutputStream',
    ShardCount=1
)

print(cos)

### Input Generator

In [ ]:
import datetime
import json
import random
import boto3

def get_data():
    return {
        'EVENT_TIME': datetime.datetime.now().isoformat(),
        'TICKER': random.choice(['AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV']),
        'PRICE': round(random.random() * 100, 2)}


def generate(stream_name, kinesis_client):
    while True:
        data = get_data()
        print(data)
        kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(data),
            PartitionKey="partitionkey")

In [ ]:
# Run it
generate("ExampleInputStream", kinesis)

### Application Code File

Compile the getting started code, create a bucket, and upload it to the bucket.

e.g.

```
mvn package
aws s3api create-bucket --bucket dskdaj-getting-started
aws s3 cp target/aws-kinesis-analytics-java-apps-1.0.jar s3://dskdaj-getting-started/getting-started.jar
```

In [ ]:
code_bucket='dskdaj-getting-started'
jar_key='getting-started.jar'

### IAM Role and Policy

Here's the generated policy as the starting point for an app named scratch.

The kinesis-analytics-scratch-us-east-1 role `via aws iam get-role --role-name kinesis-analytics-scratch-us-east-1`

```
{
    "Role": {
        "Path": "/service-role/",
        "RoleName": "kinesis-analytics-scratch-us-east-1",
        "RoleId": "xxx",
        "Arn": "arn:aws:iam::xxx:role/service-role/kinesis-analytics-scratch-us-east-1",
        "CreateDate": "2021-07-03T16:11:50+00:00",
        "AssumeRolePolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "kinesisanalytics.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        },
        "MaxSessionDuration": 3600,
        "RoleLastUsed": {}
    }
}
```
Policies:

```
aws iam list-attached-role-policies  --role-name kinesis-analytics-scratch-us-east-1
{
    "AttachedPolicies": [
        {
            "PolicyName": "kinesis-analytics-service-scratch-us-east-1",
            "PolicyArn": "arn:aws:iam::xxx:policy/service-role/kinesis-analytics-service-scratch-us-east-1"
        }
    ]
}
```

```
aws iam get-policy --policy-arn arn:aws:iam::xxx:policy/service-role/kinesis-analytics-service-scratch-us-east-1
{
    "Policy": {
        "PolicyName": "kinesis-analytics-service-scratch-us-east-1",
        "PolicyId": "xxx",
        "Arn": "arn:aws:iam::xxx:policy/service-role/kinesis-analytics-service-scratch-us-east-1",
        "Path": "/service-role/",
        "DefaultVersionId": "v1",
        "AttachmentCount": 1,
        "PermissionsBoundaryUsageCount": 0,
        "IsAttachable": true,
        "CreateDate": "2021-07-03T16:11:50+00:00",
        "UpdateDate": "2021-07-03T16:11:50+00:00",
        "Tags": []
    }
}
```

aws iam get-policy-version --policy-arn arn:aws:iam::xxx:policy/service-role/kinesis-analytics-service-sample-us-east-1 --version-id v1

```
{
    "PolicyVersion": {
        "Document": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Sid": "ReadCode",
                    "Effect": "Allow",
                    "Action": [
                        "s3:GetObject",
                        "s3:GetObjectVersion"
                    ],
                    "Resource": [
                        "arn:aws:s3:::kinesis-analytics-placeholder-s3-bucket/kinesis-analytics-placeholder-s3-object"
                    ]
                },
                {
                    "Sid": "ListCloudwatchLogGroups",
                    "Effect": "Allow",
                    "Action": [
                        "logs:DescribeLogGroups"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-east-1:xxx:log-group:*"
                    ]
                },
                {
                    "Sid": "ListCloudwatchLogStreams",
                    "Effect": "Allow",
                    "Action": [
                        "logs:DescribeLogStreams"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-east-1:xxx:log-group:/aws/kinesis-analytics/sample:log-stream:*"
                    ]
                },
                {
                    "Sid": "PutCloudwatchLogs",
                    "Effect": "Allow",
                    "Action": [
                        "logs:PutLogEvents"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-east-1:xxx:log-group:/aws/kinesis-analytics/sample:log-stream:kinesis-analytics-log-stream"
                    ]
                }
            ]
        },
        "VersionId": "v1",
        "IsDefaultVersion": true,
        "CreateDate": "2021-07-04T15:21:25+00:00"
    }
}
```


In [ ]:
assume_role_policy_document="""{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "kinesisanalytics.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
"""

In [ ]:
cr = iam.create_role(
    RoleName='kda-sample-role',
    Path='/service-role/',
    AssumeRolePolicyDocument=assume_role_policy_document
)

print(cr)

In [ ]:
import os
account_no = os.environ['PRODUCER_ACCOUNT_NO']

In [ ]:
policy_document="""{
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Sid": "ReadCode",
                    "Effect": "Allow",
                    "Action": [
                        "s3:GetObject",
                        "s3:GetObjectVersion"
                    ],
                    "Resource": [
                        "arn:aws:s3:::dskdaj-getting-started/*"
                    ]
                },
                {
                    "Sid": "ListCloudwatchLogGroups",
                    "Effect": "Allow",
                    "Action": [
                        "logs:DescribeLogGroups"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-east-1:""" + account_no + """:log-group:*"
                    ]
                },
                {
                    "Sid": "ListCloudwatchLogStreams",
                    "Effect": "Allow",
                    "Action": [
                        "logs:DescribeLogStreams"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-east-1:""" + account_no + """:log-group:/aws/kinesis-analytics/sample:log-stream:*"
                    ]
                },
                {
                    "Sid": "PutCloudwatchLogs",
                    "Effect": "Allow",
                    "Action": [
                        "logs:PutLogEvents"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-east-1:""" + account_no + """:log-group:/aws/kinesis-analytics/sample:log-stream:kinesis-analytics-log-stream"
                    ]
                }
            ]
}"""

In [ ]:
print(policy_document)

In [ ]:
cp = iam.create_policy(
    PolicyName='kda-sample-app',
    Path='/service-role/',
    PolicyDocument=policy_document
)

print(cp)

In [ ]:
ap = iam.attach_role_policy(
    RoleName='kda-sample-role',
    PolicyArn="arn:aws:iam::{}:policy/service-role/kda-sample-app".format(account_no)
)

## Clean Up

#### IAM

In [ ]:
policy_arn="arn:aws:iam::{}:policy/service-role/kda-sample-app".format(account_no)
print(policy_arn)

In [ ]:
iam.detach_role_policy(
    RoleName='kda-sample-role',
    PolicyArn=policy_arn
)

In [ ]:
iam.delete_policy(
    PolicyArn=policy_arn
)

In [ ]:
iam.delete_role(
    RoleName='kda-sample-role'
)

#### Streams

In [ ]:
kinesis.delete_stream(
    StreamName='ExampleInputStream'
)

In [ ]:
kinesis.delete_stream(
    StreamName='ExampleOutputStream'
)

In [ ]:
kinesis.list_streams()